In [1]:
import re

In [3]:
class BioChemPars():
    def __init__(self, name, path):
        """
        создаем для класса сущность, а также путь
        к файлу, в котором в дальнейшем будем ее искать
        """

        # сущность значение которой будем искать
        # на всякий случай применяем .lower() читоб исключить неправильный ввод
        self.entity_name = name.lower()

        # чтение файла возвращающее его содержание в формате str
        with open(path, encoding='utf-8', errors='ignore') as file:
            self.txt = file.read()

        self.txt = self.txt.lower()  # вводим текст в нижний регистр для удобства

        # специальный словарь для всех критериев измеряющихся не в ммоль/л
        self.special_dict = {'креатинин': 'мкмоль/л',
                             'билирубин': 'мкмоль/л',
                             'ферритин': 'нг/мл',
                             'с-реактивный белок': 'мг/л',
                             'аспартатаминотрансфераза': 'ед/л',
                             'аланинаминотрансфераза': 'ед/л',
                             'амилаза': 'ед/л',
                             'альбумины': 'г/л'
                             }

        # пока что "пустой" словарь референсов
        self.reference = []

        # пока что пустое значение
        self.znach = []

    def parse_text(self):
        """
        ищем сущность и ее значение в тексте
        """

        try:
            look_for_tag = self.entity_name + '(.+?);'  # ищем все что после сущности и до знака ';'
            look_for_res = re.findall(look_for_tag, self.txt)

            # для ЛДГ специальные параметры тк только одна граница референса, а также единицы измерения
            if self.entity_name == 'лактатдегидрогеназа':
                self.reference = 380
                split_param = 'ед/л'

            # для КК специальные параметры тк только одна граница референса, а также единицы измерения
            if self.entity_name == 'креатинкиназа':
                self.reference = 140
                split_param = 'ед/л'

            # почему-то далеко не везде есть референс, поэтому надо будет добавить
            if self.entity_name == 'общий белок':
                self.reference = [60, 80]
                split_param = 'г/л'

            # для всех остальных без одной границы референса задаем специальные единицы измерения
            if self.entity_name in self.special_dict.keys():
                split_param = self.special_dict[self.entity_name]


            # для всех остальных без одной границы референса и специальных единиц измерения (т.е. ммоль/л)
            else:
                split_param = 'ммоль/л'

            if self.entity_name != 'общий белок':  # отсеиваем шакальный общий белок тк он без референса почему-то
                for res in look_for_res:
                    znach, reference, nothing = res.split(
                        split_param)  # ед. измерения разделяет текст на значение, референс и []

                    znach = re.sub('[а-я]', '', znach)  # убираем лишние буквы (если есть)
                    znach = re.sub(' ', '', znach)  # убираем пробелы

                    if ',' in znach:  # заменяем запятую на точку чтоб перевести значение в float
                        znach = re.sub(',', '.', znach)
                        self.znach.append(float(znach))
                    else:
                        self.znach.append(float(znach))
            else:  # возвращаемся к общему белку
                for res in look_for_res:
                    znach = re.sub('г/л', '', res)
                    znach = re.sub(' ', '', znach)
                    self.znach.append(float(znach))

            # можно по последнему результату сделать референс тк он все равно везде одинаковый
            if self.entity_name not in ['общий белок', 'креатинкиназа', 'лактатдегидрогеназа']:

                # учитываем что может не найти значения (например, ферритин есть не у всех пациентов)
                if look_for_res != []:
                    znach, reference, nothing = look_for_res[0].split(split_param)
                    reference = re.sub(r'[(]|[)]', '', reference)  # убираем ()
                    reference_1, reference_2 = reference.split('-')  # убираем дефис из референса

                    self.reference.append(float(reference_1))  # добавляем нижнюю границу референса
                    self.reference.append(float(reference_2))  # добавляем верхнюю границу референса


        # могут быть моменты а ля "Глюкоза: отрицательная"
        except ValueError:
            self.znach = []


    def compare_to_reference(self, a):
        """
        сравниваем значение сущности с референсом
        """
    
        # Специальные условия для ЛДГ и КК тк у них только верхняя граница
        if self.entity_name in ['креатинкиназа', 'лактатдегидрогеназа']:
            if a <= self.reference:
                return 'норма'
            else:
                return 'отклонение'
    
                # Для всех остальных сущностей классические верхние и нижние границы
        else:
            if self.reference[0] <= a <= self.reference[1]:
                return 'норма'
            else:
                return 'отклонение'

    
    def display(self):
        """
        выдаем результаты (если есть)
        """
    
        if self.znach == []:
            print(f'{self.entity_name.upper()} РЕЗУЛЬТАТЫ')
            print('Не найдено. Проверьте правильность ввода или наличие исследования в карте пациента.')
    
        else:
            # Выводим результаты в три линии (титульник, шапка и собственно сами результаты в формате
            # порядковый номер - результат - норма/отклонение)
    
            print(f'{self.entity_name.upper()} РЕЗУЛЬТАТЫ')
            print('№  значение  референс')
    
            for el in self.znach:
                ind = self.znach.index(el) + 1
                compare = self.compare_to_reference(el)
                print(f'{ind}. {el}        {compare}')

In [4]:
i_want_to_see = ['общий белок', 'мочевина', 'креатинин', 'билирубин', 'глюкоза',
                 'аспартатаминотрансфераза', 'аланинаминотрансфераза', 'амилаза', 'калий',
                 'хлориды', 'с-реактивный белок', 'ферритин', 'альбумины']


path = 'file (1091).txt'

for el in i_want_to_see:
    test_BP = BioChemPars(el, path)
    test_BP.parse_text()
    test_BP.display()
    print(' ')

ОБЩИЙ БЕЛОК РЕЗУЛЬТАТЫ
№  значение  референс
1. 73.0        норма
2. 47.0        отклонение
 
МОЧЕВИНА РЕЗУЛЬТАТЫ
№  значение  референс
1. 6.8        норма
2. 5.5        норма
 
КРЕАТИНИН РЕЗУЛЬТАТЫ
№  значение  референс
1. 102.0        норма
2. 64.0        норма
 
БИЛИРУБИН РЕЗУЛЬТАТЫ
№  значение  референс
1. 17.0        норма
2. 11.6        норма
 
ГЛЮКОЗА РЕЗУЛЬТАТЫ
№  значение  референс
1. 9.2        отклонение
2. 12.5        отклонение
 
АСПАРТАТАМИНОТРАНСФЕРАЗА РЕЗУЛЬТАТЫ
№  значение  референс
1. 30.0        норма
2. 14.0        норма
 
АЛАНИНАМИНОТРАНСФЕРАЗА РЕЗУЛЬТАТЫ
№  значение  референс
1. 42.0        норма
2. 41.0        норма
 
АМИЛАЗА РЕЗУЛЬТАТЫ
№  значение  референс
1. 42.0        норма
 
КАЛИЙ РЕЗУЛЬТАТЫ
№  значение  референс
1. 4.61        норма
 
ХЛОРИДЫ РЕЗУЛЬТАТЫ
№  значение  референс
1. 106.0        норма
2. 91.0        отклонение
 
С-РЕАКТИВНЫЙ БЕЛОК РЕЗУЛЬТАТЫ
Не найдено. Проверьте правильность ввода или наличие исследования в карте пациента.
 
ФЕРРИТИН РЕЗУЛЬТАТ

In [5]:
path = 'file (1072).txt'

for el in i_want_to_see:
    test_BP = BioChemPars(el, path)
    test_BP.parse_text()
    test_BP.display()
    print(' ')

ОБЩИЙ БЕЛОК РЕЗУЛЬТАТЫ
№  значение  референс
1. 64.0        норма
2. 59.0        отклонение
 
МОЧЕВИНА РЕЗУЛЬТАТЫ
№  значение  референс
1. 4.5        норма
2. 4.6        норма
 
КРЕАТИНИН РЕЗУЛЬТАТЫ
№  значение  референс
1. 78.0        норма
1. 78.0        норма
 
БИЛИРУБИН РЕЗУЛЬТАТЫ
№  значение  референс
1. 13.2        норма
2. 11.2        норма
 
ГЛЮКОЗА РЕЗУЛЬТАТЫ
Не найдено. Проверьте правильность ввода или наличие исследования в карте пациента.
 
АСПАРТАТАМИНОТРАНСФЕРАЗА РЕЗУЛЬТАТЫ
№  значение  референс
1. 24.0        норма
2. 26.0        норма
 
АЛАНИНАМИНОТРАНСФЕРАЗА РЕЗУЛЬТАТЫ
№  значение  референс
1. 23.0        норма
2. 29.0        норма
 
АМИЛАЗА РЕЗУЛЬТАТЫ
Не найдено. Проверьте правильность ввода или наличие исследования в карте пациента.
 
КАЛИЙ РЕЗУЛЬТАТЫ
№  значение  референс
1. 4.1        норма
2. 4.2        норма
 
ХЛОРИДЫ РЕЗУЛЬТАТЫ
№  значение  референс
1. 108.0        норма
2. 100.0        норма
 
С-РЕАКТИВНЫЙ БЕЛОК РЕЗУЛЬТАТЫ
Не найдено. Проверьте правильность вво